### A partir de cette version, on veut plutôt sélectionner les **catégories significatives** avec un "grand" nombre d'exposés des faits associé.

In [1]:
import pandas as pd
import numpy as np
import math
import random
import re
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame. 
                                           # no warning message and no exception is raised
import spacy
nlp = spacy.load('fr')
spacy.load('en')

import gensim.summarization.bm25 as bm25
from textblob import TextBlob as tb

import nltk 
import operator

A **DataFrame** is a two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns). Arithmetic operations align on both row and column labels. Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

Ci-dessous, efs est un DataFrame.
L'argument header=0 indique que les noms des colonnes se trouvent sur la première ligne (d'indice 0).

In [2]:
efs = pd.read_csv('/home/alina/UNIFR/TM/random/ExposesDesFaits/exposes_faits_preproc_TEMOIN.csv', header=0, sep=';')
efs.head(5)

,EXPOSES,CATEGORIE
0,Une trace de mazout sur env 20 mètres. Pompier...,POLLUTION - HYDROCARBURES
1,avis prudence météo,FEUX A ECLIPSES - 45 TOURS
2,Un renard sur la voie gauche.,ANIMAUX
3,Interpellation de M. Ndue GESHTEJA,INDIVIDU - RECHERCHE
4,Bras cassé suite glissade sur le verglas.,DEMANDE - D'AMBULANCE


In [3]:
print(type(efs))

<class 'pandas.core.frame.DataFrame'>


**Le nombre d'enregistrements** de la DataFrame efs

In [4]:
len(efs)

158071

## Manipulations des enregistrements

Les enregistrements qui ont comme **CATEGORIE** ANIMAUX ou animaux ou Animaux

In [5]:
efs_categorie_animaux = efs.query('CATEGORIE == "ANIMAUX" or CATEGORIE == "animaux" or CATEGORIE == "Animaux"')
efs_categorie_animaux.head(5)

,EXPOSES,CATEGORIE
2,Un renard sur la voie gauche.,ANIMAUX
11,1 chien dans la Renault Kangoo blanche (F) AN8...,ANIMAUX
86,retrouvé un chat en mauvais état,ANIMAUX
95,un serpent vivant au milieu de la route,ANIMAUX
101,Des chèvres sur la route.,ANIMAUX


Les indices des enregistrements ayant comme **CATEGORIE** ANIMAUX ou animaux ou Animaux

In [6]:
efs_categorie_animaux.index

Int64Index([     2,     11,     86,     95,    101,    184,    190,    208,
               230,    295,
            ...
            157689, 157696, 157747, 157777, 157833, 157892, 157911, 157937,
            158016, 158050],
           dtype='int64', length=6847)

ATTENTION : le dernier enregistrement de la DataFrame efs est bien len(data) car on n'a pas effacé des enregistrements (cas où les indices n'auraient pas été mis à jour).

Le **dernier** enregistrement de la DataFrame **efs**.

In [7]:
print(efs.iloc[-1])

EXPOSES      4 paires de skis sur la chaussée.
CATEGORIE                          AR - DIVERS
Name: 158070, dtype: object


Les **derniers** enregistrements de la DataFrame **efs**

In [8]:
print(efs.tail(3))

                                                  EXPOSES  \
158068  Adj IMSAND GDM/Protection des personnalités di...   
158069  Demande de venir sur place. N'arrivons pas à d...   
158070                  4 paires de skis sur la chaussée.   

                     CATEGORIE  
158068             INFORMATION  
158069  DEMANDE - D'ASSISTANCE  
158070             AR - DIVERS  


Une **tranche** d'enregistrements (entre les indices 50500 et 50505) de la DataFrame efs

In [9]:
efs.loc[50500:50505]

,EXPOSES,CATEGORIE
50500,1 arbre en travers de la route,INTEMPERIE
50501,Intervention spontanée suite à bruits audibles...,DEMANDE - D'ASSISTANCE
50502,A trouvé des cartouches d'arme dans le corrido...,OBJETS - TROUVES
50503,"en face du camping de Vidy à 400m du bord, 2 a...",ACCIDENT - DE NAVIGATION / NAUFRAGE - NAVIGATE...
50504,Pour un homme qui sème le scandale et ne veux ...,INDESIRABLE
50505,Retrouve une voiture à plaques Valaisannes pla...,ACCIDENT - DE CIRCULATION - AVEC FUITE


**L'enregistrement** avec l'indice 50500 de la DataFrame efs, en tant qu'objet **Series**

In [10]:
print(type(efs.loc[50500]))
efs.loc[50500]

<class 'pandas.core.series.Series'>


EXPOSES      1 arbre en travers de la route
CATEGORIE                        INTEMPERIE
Name: 50500, dtype: object

**L'exposé** (premier indice donc de valeur 0) de l'enregistrement avec l'indice 50500 de la DataFrame efs

In [11]:
efs.loc[50500][0]

'1 arbre en travers de la route'

**La catégorie** (deuxième indice donc de valeur 1) de l'enregistrement avec l'indice 50500 de la DataFrame efs

In [12]:
efs.loc[50500][1]

'INTEMPERIE'

On obtient ci-dessous les indices des lignes (donc les étiquettes des lignes qui chez nous sont des entiers) de la **DataFrame**.

In [13]:
efs.index

RangeIndex(start=0, stop=158071, step=1)

La liste des axes de la **DataFrame**.

En particulier, on peut y trouver les noms (les étiquettes) des colonnes.

In [14]:
efs.axes

[RangeIndex(start=0, stop=158071, step=1),
 Index(['EXPOSES', 'CATEGORIE'], dtype='object')]

Si jamais on veut intervertir l'ordre des catégories

In [15]:
efs_mix = efs[['CATEGORIE','EXPOSES']]

In [16]:
efs_mix.head(5)

,CATEGORIE,EXPOSES
0,POLLUTION - HYDROCARBURES,Une trace de mazout sur env 20 mètres. Pompier...
1,FEUX A ECLIPSES - 45 TOURS,avis prudence météo
2,ANIMAUX,Un renard sur la voie gauche.
3,INDIVIDU - RECHERCHE,Interpellation de M. Ndue GESHTEJA
4,DEMANDE - D'AMBULANCE,Bras cassé suite glissade sur le verglas.


### Catégories

Les catégories des premiers enregistrements 

In [17]:
print(type(efs['CATEGORIE']))
efs['CATEGORIE'].head(3)

<class 'pandas.core.series.Series'>


0     POLLUTION - HYDROCARBURES
1    FEUX A ECLIPSES - 45 TOURS
2                       ANIMAUX
Name: CATEGORIE, dtype: object

Les catégories des derniers enregistrements

In [18]:
efs['CATEGORIE'].tail(3)

158068               INFORMATION
158069    DEMANDE - D'ASSISTANCE
158070               AR - DIVERS
Name: CATEGORIE, dtype: object

Les catégories de tous les enregistrements (donc les éléments de la colonne CATEGORIE) de la DataFrame efs en tant qu'objet de type pandas.core.series.Series et aussi liste de strings

In [19]:
categories = efs['CATEGORIE']
#categories = efs.CATEGORIE
print(type(categories))
l_categories = list(categories)
l_categories[0:5:1]

<class 'pandas.core.series.Series'>


['POLLUTION - HYDROCARBURES',
 'FEUX A ECLIPSES - 45 TOURS',
 'ANIMAUX',
 'INDIVIDU - RECHERCHE',
 "DEMANDE - D'AMBULANCE"]

Les catégories SANS doublons comme set et liste
On transforme la Series "categories" en un set nommé "set_categories" pour éliminer les doublons

In [20]:
set_categories = set(categories)
print(type(set_categories))
print(len(set_categories))
#set_categories

<class 'set'>
431


Quelques exemples de catégories choisis aléatoirement

In [21]:
set(random.sample(set_categories,5))

{'ABUS - DE CONFIANCE',
 "ALARME MISE EN TEST - 421250 - MISTEST  - FONDATION DE L'HERMITAGE (MUSEE)",
 'DISPARITION - PERSONNE AGEE',
 'RECHERCHE - ALARME - VAUD',
 "VOL - A L'ASTUCE"}

En affichant toutes les catégories avec l'instruction commentée ci-dessous, on constate qu'il y a encore des enregistrements pour lesquels la valeur de la CATEGORIE est en fait un exposé des faits.
Par la suite, on efface ces faux enregistrements. 

In [22]:
list(set_categories)[0:11]

['PIEGE - PIEGES SMS-GTM - 121  - Alarmes piège',
 'FUITE - HOPITAL',
 'PIEGE - PIEGES SMS-GTM - 129  - Alarmes piège',
 'ALARME GTC  CHAUFFAGE-VENTILATION - ESPA - 192  - Alarmes bâtiment ESPA',
 'ALARME MISE EN TEST - 420627 - MISTEST  - GRANDS MAGASINS MANOR - MAUS & CIE LAUSANNE (MAGASIN)',
 'EPIZOOTIE - PANDEMIE',
 'AR - ACCIDENT',
 'ALARME - CONDUITE SPEN',
 'AR - INCENDIE',
 'COLLABORATION - FRANCO-SUISSE.  - DEMANDE FRANCAISE',
 'ACCIDENT - DE TIR']

On veut éliminer les enregistrements dont la CATEGORIE ne commence pas directement par une majuscule.

On stocke dans set_categories_to_delete les valeurs des fausses catégories.

In [23]:
set_categories_to_delete = set(   filter(lambda x:not x[0].isupper(), set_categories)    )
set_categories_to_delete

{" Des véhicules gênent l'accès.",
 ' GOSENDE passera dès demain dans un poste.',
 ' Toute la journée',
 ' VD 216322 Citroën Berlingo blanche avec inscription Alvazzi. ',
 ' du 0337ème positif à domicile.',
 " il lui a répondu que c'était pour la famille. Trouvait cela suspect, mais n'a pas pensé à nous appeler de suite.",
 ' jogging) qui est parti avec un trottinnette direction Chauderon.',
 ' mi-trottoir en direction de la Maladière.',
 ' puis jets de bouteilles contre police.',
 ': homme, chauve, Lunette noires, veste cuire marron, sac à dos.'}

On efface les enregistrements qui sont faux et on rénumérote les enregistrements après effacement.

Attention: une même fausse catégorie peut apparaître pour plusieurs enregistrements => l'intérêt de la boucle for interne

In [24]:
for cat in set_categories_to_delete:
    indices = efs.index[efs['CATEGORIE']==cat].tolist()
    #print(indices)
    for ind in indices:
        #print(ind)
        efs.drop(ind, inplace=True)
efs.reset_index(inplace=True)
efs.index

RangeIndex(start=0, stop=158058, step=1)

Les nouvelles catégories restantes (donc justes).

In [25]:
categories = efs['CATEGORIE']
#categories = efs.CATEGORIE
set_categories = set(categories)
print(len(set_categories))

421


## Catégories comme nouveaux DataFrame

On peut obtenir un nouveau DataFrame correspondant seulement aux enregistrements dont la CATEGORIE a une certaine valeur.

On considère par exemple la catégorie d'indice 22.

In [26]:
l_categories = list(set_categories)
l_categories[22]

'MORT - NATURELLE'

Les indices des enregistrements dans la DataFrame globale efs correspondant aux enregistrements dont la catégorie a une certaine valeur

In [27]:
indices_22 = efs.index[efs['CATEGORIE']==l_categories[22]].tolist()
print(len(indices_22))
indices_22[0:10]

286


[242, 801, 2120, 2365, 4132, 5695, 6757, 8216, 8284, 8505]

Le nouveau DataFrame contenant seulement les enregistrements dont la CATEGORIE a une certaine valeur.

In [28]:
efs_categorie_22 = efs.loc[efs['CATEGORIE']==l_categories[22]]
print(len(efs_categorie_22))
efs_categorie_22.head(3)

286


,index,EXPOSES,CATEGORIE
242,242,Une femme d'un certain âge est tombée après êt...,MORT - NATURELLE
801,801,Dr Sellrath se trouve au domicile de Mme Ganty...,MORT - NATURELLE
2120,2120,Le docteur du cabinet MEDICAVO (au numéro 7) a...,MORT - NATURELLE


In [29]:
type(efs['CATEGORIE']==l_categories[22])

pandas.core.series.Series

In [30]:
efs_categorie_22.drop(['index'], axis=1, inplace=True)
efs_categorie_22.head(3)

,EXPOSES,CATEGORIE
242,Une femme d'un certain âge est tombée après êt...,MORT - NATURELLE
801,Dr Sellrath se trouve au domicile de Mme Ganty...,MORT - NATURELLE
2120,Le docteur du cabinet MEDICAVO (au numéro 7) a...,MORT - NATURELLE


In [31]:
efs_categorie_22.reset_index(inplace=True, drop=True)
efs_categorie_22.head(3)

,EXPOSES,CATEGORIE
0,Une femme d'un certain âge est tombée après êt...,MORT - NATURELLE
1,Dr Sellrath se trouve au domicile de Mme Ganty...,MORT - NATURELLE
2,Le docteur du cabinet MEDICAVO (au numéro 7) a...,MORT - NATURELLE


In [32]:
l_cat_tri = sorted(l_categories)
l_cat_tri[:5]

['ABUS - DE CONFIANCE',
 'ABUS - DU TELEPHONE',
 'ACCIDENT',
 'ACCIDENT - AVEC DES EXPLOSIFS',
 "ACCIDENT - D'ASCENCEUR"]

On crée une liste réduite de catégories qui seront utilisées par la suite, à savoir les catégories :
- ACCIDENT
- ACCIDENT - DE TRAVAIL
- ACCIDENT - DE CIRCULATION - AVEC ANIMAUX
- ACCIDENT - DE CIRCULATION - AVEC BLESSE
- INONDATION

In [33]:
categories_red = ['ACCIDENT', 'ACCIDENT - DE TRAVAIL', 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL', 
                      'ACCIDENT - DE CIRCULATION - AVEC BLESSE', 'INONDATION']

La variable efs_par_cat est une liste dont chaque élément est un DataFrame qui correspond à une certaine catégorie.

Pour le cas général, il suffit de remplacer dans le for ci-dessous **categories_red** par **categories**.

In [34]:
efs_par_cat = []
for cat in categories_red:
    df = efs.query('CATEGORIE == @cat')
    df.drop(['index'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)
    print(cat, ' ', len(df))
    efs_par_cat.append(df)

ACCIDENT   90
ACCIDENT - DE TRAVAIL   426
ACCIDENT - DE CIRCULATION - AVEC ANIMAL   1714
ACCIDENT - DE CIRCULATION - AVEC BLESSE   1960
INONDATION   476


In [35]:
efs_par_cat[0].head(3)

,EXPOSES,CATEGORIE
0,"Une fille serait tombé au sol, saigne de la bo...",ACCIDENT
1,Une femme a été blessée par une tente qui s'es...,ACCIDENT
2,Une grue a cassé la ligne de contact des troll...,ACCIDENT


## Dictionnaire

Par la suite, on va travailler avec un dictionnaire **dico** dont les éléments sont des couples où la clé est une catégorie et la valeur est une liste de strings correspondant aux exposés des faits associés à cette catégorie.

On crée d'abord une fonction qui affiche les **n_cats** premiers couples d'un dictionnaire et pour chaque couple affiche sa catégorie et les **n_expos** exposés des faits associés à cette catégorie.

In [36]:
# TODO : à vérifier et à améliorer
def show(dictionnaire, n_cats, n_expos):
    i=0
    for cat,expos in dictionnaire.items():
        j=0
        print(cat,end='\n***************\n')
        for expo in expos:
            print(expo,end='\n----------------\n')
            j+=1
            if j == n_expos:
                break 
        i+=1
        if i == n_cats:
            break

On crée un **dictionnaire** qui a comme clés les catégories et comme valeurs une liste avec les exposés des faits correspondants (à chaque catégorie, on associe la liste avec ses exposés des faits).

Pour le moment, la variable efs_par_cat contient seulement les DataFrame pour les 5 catégories qu'on utilise pour exemplifier notre démarche.

In [37]:
dico = {}
for df in efs_par_cat:
    #expos = list(data.query('CATEGORIE == @cat')['EXPOSES'].values)
    #print(df['CATEGORIE'][0])
    #print(type(list(df['EXPOSES'].values)))
    dico[df['CATEGORIE'][0]] = list(df['EXPOSES'].values)

In [38]:
show(dico,5,2)

ACCIDENT
***************
Une fille serait tombé au sol, saigne de la bouche. Pas plus de précisions. 
----------------
Une femme a été blessée par une tente qui s'est envolée, ambulance sur place et acheminent la blessée à l'Hôpital de Montreux.
----------------
ACCIDENT - DE TRAVAIL
***************
Un homme fait une hémorragie au niveau d'un avant-bras. SMUR et ambulance engagés
----------------
1 chasse-neige a écrasé un piéton
----------------
ACCIDENT - DE CIRCULATION - AVEC ANIMAL
***************
1 auto & 1 cervidé, animal à priori "DCD" en bordure de route. La voiture Opel Corsa, blanche, VD454348, conduite par la concubine de l'informateur n'a pas de dommage et a quitté les lieux !
----------------
A heurté du gibier ce jour vers 0800, a mis la bête sur le côté et a avisé le garde-faune qui prendra la bête en charge. 
----------------
ACCIDENT - DE CIRCULATION - AVEC BLESSE
***************
Accident auto VD-563'166  Audi Q3 blanc - auto VD-262'160. Informateur se plaint de douleu

**efs_par_cat_global** est une liste des DataFrame où chaque DataFrame a la même valeur pour la colonne catégorie

In [56]:
efs_par_cat_global = []
for cat in set_categories:
    df = efs.query('CATEGORIE == @cat')
    df.drop(['index'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)
    efs_par_cat_global.append(df)
print(len(efs_par_cat_global))    
efs_par_cat_global[21][0:5]
#efs_par_cat_global[21]
#print(efs_par_cat_global)

421


,EXPOSES,CATEGORIE
0,5BF6B042-1456-40CB-8BA2-65CE454CA2C2,PIEGE - PIEGES SMS-GTM - 134 - Alarmes piège


**dico_global** est un dictionnaire dont les clés sont les catégories distinctes et les valeurs des listes d'exposés associés à ces catégories 

In [57]:
dico_global = {}
for df in efs_par_cat_global:
    dico_global[df['CATEGORIE'][0]] = list(df['EXPOSES'].values)
show(dico_global,5,2)    

PIEGE - PIEGES SMS-GTM - 121  - Alarmes piège
***************
527FE30B-AE42-4B59-AB5F-2EBBDA191294
----------------
FUITE - HOPITAL
***************
Mme Odette Margot, Clair-Joly à Lutry (86 ans, cheveux noirs, queue de cheval à gauche, habillée en noir, chausson) a quitté L'Hôpital Nestlé. Lors du tél. elle a été récupérée à Victor-Ruffy 51 par un passant et amené à son domicile, 3ème étage, chez Poncelet.
----------------
Une patiente en décompensation (auto et hétéro agression) qui est partie du PLI vers 1700. Signalement: corpulence forte (90kg), chx bruns, peau claire, leggins blanc, haut foncé épaules dénudées. REARDON Marguerite (Margo) 24.06.1981 DAVEL 21 Lsne
----------------
PIEGE - PIEGES SMS-GTM - 129  - Alarmes piège
***************
EE774088-0087-48E2-90C7-A864571EE204
----------------
ALARME GTC  CHAUFFAGE-VENTILATION - ESPA - 192  - Alarmes bâtiment ESPA
***************
Alarme chauffage-ventilation
----------------
Alarme chauffage-ventilation
----------------
ALARME MISE

**dico_categories_nb_expos** est un dictionnaire dont les clés sont les catégories et les valeurs les nombres d'exposés par catégories

In [41]:
dico_categories_nb_expos = {}
for key,value in dico_global.items():
    #print(key, ' ', len(value))
    dico_categories_nb_expos[key] = len(value)

On peut afficher les catégories avec les nombres d'exposés associés à chacune d'entre elles en ordre croissant

In [42]:
for key,value in sorted(dico_categories_nb_expos.items(), key=operator.itemgetter(1), reverse=True):
    print(key,'   ', value)

DEMANDE - D'ASSISTANCE     17968
TAPAGE NOCTURNE     7182
ACCIDENT - DE CIRCULATION - DEGATS MATERIEL     7091
ANIMAUX     6847
VOL - PAR EFFRACTION     6657
AR - DIVERS     5617
INDIVIDU - SUSPECT     5261
INFRACTION - LCR     5141
INDIVIDU - PERTURBE     4727
AR - PANNE - VEHICULE     3318
OPERATION     3149
DOMMAGES - A LA PROPRIETE     2867
APPREHENSION / ARRESTATION     2510
BAGARRE     2496
CIRCULATION - ROUTIERE     2447
AR - ACCIDENT     2337
REVOCATION     2317
LITIGE     2296
MINEUR - IMPLIQUE     2287
INDESIRABLE     2265
ACCIDENT - DE CIRCULATION - AVEC FUITE     2213
VEHICULE - SUSPECT     2199
INCENDIE     2085
ACCIDENT - DE CIRCULATION - AVEC BLESSE     1960
VOL     1918
DROGUE     1813
ACCIDENT - DE CIRCULATION - AVEC ANIMAL     1714
DEMANDE - D'AMBULANCE     1662
VOL - A L'ETALAGE     1611
FUITE - D'UN LIEU DE PLACEMENT     1466
VIOLENCE - DOMESTIQUE     1437
TENTATIVE     1369
COLLABORATION - INTERPOLICE     1340
DEMANDE - IDENTIFICATION     1262
VEHICULE     1164
TRO

On peut afficher les catégories avec les nombres d'exposés associés qui se trouvent dans une plage donnée 

In [62]:
nb_cats = 0
val_min = 1000
val_max = 100000
nb_expos_tot = 0
for key,value in sorted(dico_categories_nb_expos.items(), key=operator.itemgetter(1), reverse=True):
    if value >= val_min and value <= val_max:  
        print(key,'   ', value)
        nb_cats+=1
        nb_expos_tot+=value
print('Nombre catégories dans la plage : ', nb_cats)
print('Nombre total d\'exposés dans la plage : ', nb_expos_tot)

DEMANDE - D'ASSISTANCE     17968
TAPAGE NOCTURNE     7182
ACCIDENT - DE CIRCULATION - DEGATS MATERIEL     7091
ANIMAUX     6847
VOL - PAR EFFRACTION     6657
AR - DIVERS     5617
INDIVIDU - SUSPECT     5261
INFRACTION - LCR     5141
INDIVIDU - PERTURBE     4727
AR - PANNE - VEHICULE     3318
OPERATION     3149
DOMMAGES - A LA PROPRIETE     2867
APPREHENSION / ARRESTATION     2510
BAGARRE     2496
CIRCULATION - ROUTIERE     2447
AR - ACCIDENT     2337
REVOCATION     2317
LITIGE     2296
MINEUR - IMPLIQUE     2287
INDESIRABLE     2265
ACCIDENT - DE CIRCULATION - AVEC FUITE     2213
VEHICULE - SUSPECT     2199
INCENDIE     2085
ACCIDENT - DE CIRCULATION - AVEC BLESSE     1960
VOL     1918
DROGUE     1813
ACCIDENT - DE CIRCULATION - AVEC ANIMAL     1714
DEMANDE - D'AMBULANCE     1662
VOL - A L'ETALAGE     1611
FUITE - D'UN LIEU DE PLACEMENT     1466
VIOLENCE - DOMESTIQUE     1437
TENTATIVE     1369
COLLABORATION - INTERPOLICE     1340
DEMANDE - IDENTIFICATION     1262
VEHICULE     1164
TRO

**main_dico** est un dictionnaire dont les clés sont les catégories distinctes **avec, par exemple, plus de 1000 exposés associés** et les valeurs sont des listes d'exposés associés à ces catégories

**l_main_cats** est une liste avec les catégories principales, i.e. avec par exemple plus de 1000 exposés associés

In [73]:
main_dico = {}
l_main_cats = []
val_min = 1000
val_max = 100000
nb_expos_tot = 0
for key,value in sorted(dico_categories_nb_expos.items(), key=operator.itemgetter(1), reverse=True):
    if value >= val_min and value <= val_max: 
        main_dico[key] = dico_global[key]
        l_main_cats.append(key)
        nb_expos_tot+=value
print('Le nombre de catégories dans le main_dico : ' + str(len(main_dico)) + '\n')
print('Nombre total d\'exposés dans le main_dico : ' + str(nb_expos_tot) + '\n')
print(l_main_cats, '\n')
#print(l_main_cats[0], '\n')
#show(main_dico, 40, 1)

Le nombre de catégories dans le main_dico : 37

Nombre total d'exposés dans le main_dico : 122204

["DEMANDE - D'ASSISTANCE", 'TAPAGE NOCTURNE', 'ACCIDENT - DE CIRCULATION - DEGATS MATERIEL', 'ANIMAUX', 'VOL - PAR EFFRACTION', 'AR - DIVERS', 'INDIVIDU - SUSPECT', 'INFRACTION - LCR', 'INDIVIDU - PERTURBE', 'AR - PANNE - VEHICULE', 'OPERATION', 'DOMMAGES - A LA PROPRIETE', 'APPREHENSION / ARRESTATION', 'BAGARRE', 'CIRCULATION - ROUTIERE', 'AR - ACCIDENT', 'REVOCATION', 'LITIGE', 'MINEUR - IMPLIQUE', 'INDESIRABLE', 'ACCIDENT - DE CIRCULATION - AVEC FUITE', 'VEHICULE - SUSPECT', 'INCENDIE', 'ACCIDENT - DE CIRCULATION - AVEC BLESSE', 'VOL', 'DROGUE', 'ACCIDENT - DE CIRCULATION - AVEC ANIMAL', "DEMANDE - D'AMBULANCE", "VOL - A L'ETALAGE", "FUITE - D'UN LIEU DE PLACEMENT", 'VIOLENCE - DOMESTIQUE', 'TENTATIVE', 'COLLABORATION - INTERPOLICE', 'DEMANDE - IDENTIFICATION', 'VEHICULE', 'TROUBLE - DE LA TRANQUILLITE / NUISANCE', 'BRUIT'] 



**l_couples_expo_noCat** est une liste de couples où le premier élément d'un couple est un exposé des faits et le deuxième élément du même couple est l'indice de la catégorie associée à cet exposé des faits (et avec cet indice on peut trouver le nom de la catégorie comme l'élément ayant cet indice dans la liste **l_main_cats**

In [79]:
l_couples_expo_noCat = []
noCat = 0
for cat,expos in main_dico.items():
    for expo in expos:
        l_couples_expo_noCat.append((expo,noCat))
    noCat+=1
print(len(l_couples_expo_noCat), '\n')
l_couples_expo_noCat[:3]

122204 



[('Les voisins cassent tout dans la maison. Ca hurle fort. Cas récurent. ', 0),
 ('son ex Gyorgi Zoltan SZEKELY 17.11.1982, dom Genève, harcèle l\'informatrice depuis plusieurs mois. A quitté le logement il y a environ 15 mn. Serait sous MAN par la Procureur WEINGART  de Morges. Il se déplace avec Citroën break grise portant plaques genevoises. A l\'intérieur, 1 BA, noir. Sign : 180 cm, corp normale, crâne rasé, yeux bleus, veste bleu "pétant" Va contacter directement le MP à Morges pour signaler les faits. A très peu de son ex qui pourrait être violent',
  0),
 ('2 jeunes ont lancé des pives sur son balcon. Partis > entrée hôpital. un avec polo blanc et chiffre 10',
  0)]

In [81]:
print(l_couples_expo_noCat[35783])

('Une vache brune et blanche déambule sur la route. Marche direction Chapelle.', 3)


In [82]:
print(l_couples_expo_noCat[35783][0])

Une vache brune et blanche déambule sur la route. Marche direction Chapelle.


In [88]:
print('L\'exposé "', l_couples_expo_noCat[35783][0], '" a la catégorie numéro ', l_couples_expo_noCat[35783][1],
      ' qui est en fait la catégorie : "', l_main_cats[l_couples_expo_noCat[35783][1]], '".')

L'exposé " Une vache brune et blanche déambule sur la route. Marche direction Chapelle. " a la catégorie numéro  3  qui est en fait la catégorie : " ANIMAUX ".


## Les définitions des catégories

On a besoin de quelques librairies supplémentaires

In [45]:
import gensim.summarization.bm25 as bm25
from textblob import TextBlob as tb

A ce stade, la variable **dico** représente le dictionnaire avec les catégories (pour le moment les 5 catégories) comme clés et, pour chaque clé, une liste de listes (qui est la liste des exposés des faits correspondants) comme valeur.

On crée une nouvelle variable **dico_flat** qui a les catégories comme clés et, pour chaque clé, une simple liste comme valeur et cette liste contient tous les mots des exposés associés à la catégorie.

De plus, la nouvelle variable **corpus** est une liste de listes dans laquelle chaque liste interne contient tous les mots de tous les exposés pour une catégorie.

Par la suite, un **document** du **corpus** représente en fait la liste de tous les mots de tous les exposés des faits d'une catégorie (à part les "mots sans intérêt").

#TODO Nettoyer encore plus les mots sans intérêt

**spacy_stop_words** est un set qui contient les "stop words" de la langue française

In [46]:
spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS
print(type(spacy_stop_words))
print(list(spacy_stop_words)[0:10])

<class 'set'>
["s'", 'hors', 'autres', 'vivat', 'excepté', 'pu', 'ses', 'moindres', 'allaient', 'd’']


In [47]:
dico_flat = {}
corpus = []
for key,value in dico.items():
    mots = []
    for expo in value:
        blob = tb(expo.lower())
        tokens = list(blob.words)
        # on élimine les stop words
        tokens = [elem for elem in tokens if elem not in spacy_stop_words]
        # on élimine les mots contenant au moins un chiffre
        tokens = [elem for elem in tokens if not any(c.isdigit() for c in elem)]
        mots+=tokens
    corpus.append(mots)
    dico_flat[key] = mots
#dico_flat
corpus[0][0:5]

['fille', 'tombé', 'sol', 'saigne', 'bouche']

**bm25.BM25(corpus)** crée un objet de type helper class BM25 associé au corpus_exemple

In [48]:
bm = bm25.BM25(corpus)
print("Le BM25\n", bm)

Le BM25


**bm.corpus** retourne la liste de listes correspondant à l'objet bm de type BM25

In [49]:
#print("Le corpus de BM25\n", bm.corpus)

**bm.corpus_size** retourne le nombre N de documents dans le corpus associé à bm

In [50]:
print(bm.corpus_size)

5


**bm.avgdl** retourne la longueur moyenne (en mots) d'un document dans le corpus (la somme de toutes les longueurs divisée par le N nombre de documents)

In [51]:
print(bm.avgdl)

6172.2


**bm.f** retourne les terms frequencies **dans chaque document** qui précisent le nombre d'occurences (par abus de langage, la fréquence) de chaque mot (distinct) dans un document donné sous la forme d'une liste de N dictionnaires : un dictionnaire par document avec comme clés les mots distincts du document et comme valeurs les nombres d'occurences dans ce document

Ci-dessous, dans le premier dictionnaire, à la clé 'fille' correspond la valeur 4 ce qui indique que le mot "fille" apparaît 4 fois dans le premier dictionnaire, donc dans le premier document.

In [52]:
bm_list_dicos_f = bm.f
print(bm_list_dicos_f[0]['fille'])

4


On calcule maintenant les fréquences relatives de chaque mot dans les dictionnaires (où la fréquence relative d'un mot dans un document est le quotient entre le nombre d'occurences de ce mot dans le document et le nombre de mots du document).

Ci-dessous, dans le premier dictionnaire, à la clé 'fille' correspond la valeur 0.0049 ce qui indique la fréquence relative du mot "fille" dans le premier dictionnaire, donc dans le premier document.

In [53]:
bm_list_dicos_f_rel = []
for dic in bm_list_dicos_f:
    dic_rel = {}
    lon = sum(dic.values())
    for key,value in dic.items():
        dic_rel[key] = value/lon
    bm_list_dicos_f_rel.append(dic_rel)
print(bm_list_dicos_f_rel[0]['fille'])

0.004968944099378882


Comme vérification, on peut s'assurer que la somme des fréquences relatives pour un même document, par exemple le premier, vaut bien 1.

In [54]:
somme = 0
for value in bm_list_dicos_f_rel[0].values():
    somme += value
somme

1.0000000000000093

**bm.df** retourne les **document frequencies** pour **tout le corpus** qui précisent le nombre d'occurences (la fréquence, par abus de langage) de chaque mot (distinct) dans le corpus sous la forme d'un dictionnaire unique avec autant d'éléments que de **mots distincts** dans le corpus et où un élément a : 
- comme clé un **mot distinct** $mot_i$ **du corpus**
- comme valeur le nombre de documents $n(mot_i)$ contenant (au moins une fois) le mot respectif

Ci-dessous, dans le seul dictionnaire, à la clé 'fille' correspond la valeur 3 ce qui indique que le mot "fille" est apparu (au moins une fois) dans 3 documents différents.

In [55]:
bm_dico_df = bm.df
print(bm_dico_df['fille'])

3


**bm.idf** retourne les **inverse document frequencies** pour **tout le corpus** qui précisent la fréquence inverse pour tout le corpus sous la forme d'un dictionnaire unique avec autant d'éléments que de mots distincts dans le corpus et où un élément a : 
- comme clé un mot distinct **du corpus**
- comme valeur la fréquence inverse du mot respectif

La fréquence inverse d'un mot $IDF(mot_i)$ est calculée comme le logarithme naturel du quotient :

$$IDF(mot_i) = log{\frac{N-n(mot_i)+0.5}{n(mot_i)+0.5}}$$

où $N$ est le nombre total de documents dans le corpus et $n(mot_i)$ est le nombre de documents qui contiennent (au moins une fois) le $mot_i$

Ci-dessous, dans le seul dictionnaire, à la clé 'fille' correspond la valeur -0.3364... qui correspond à sa fréquence inversée.

In [56]:
bm_dico_idf = bm.idf
print(bm_dico_idf['fille'])

-0.33647223662121295


On peut aussi calculer une autre fréquence inversée d'un mot comme le logarithme naturel du quotient
$$IDF_{bis}(mot_i) = log{\frac{N}{n(mot_i)+1}}$$

où $N$ est le nombre total de documents dans le corpus et $n(mot_i)$ est le nombre de documents qui contiennent (au moins une fois) le $mot_i$

Ci-dessous, dans le seul dictionnaire, à la clé 'fille' correspond la valeur 0.2231... qui correspond à sa fréquence inversée.

In [57]:
bm_dico_idf_perso = {}
size = bm.corpus_size
for key,value in bm_dico_df.items():
    bm_dico_idf_perso[key] = math.log(size/(1+value))
print(bm_dico_idf_perso['fille'])

0.22314355131420976


Pour chaque mot de chaque document, on calcule le produit entre sa fréquence relative (afin de tenir compte du "rôle" de ce mot dans le document) et sa fréquence inverse (afin de tenir compte du "rôle" de ce mot dans le corpus).

Entre autre, grâce à la fréquence inversée, les mots qui interviennent dans toutes les définitions des catégories (et qui ne nous permettent pas de distinguer ces catégories) seront pénalisés.

In [58]:
bm_list_dicos_tfidf = []
for dic in bm_list_dicos_f_rel:
    dic_tfidf = {}
    for key,value in dic.items():
        dic_tfidf[key] = value*bm_dico_idf[key]
        #dic_tfidf[key] = value*bm_dico_idf_perso[key]
    bm_list_dicos_tfidf.append(dic_tfidf)
print(bm_list_dicos_tfidf[0]['fille'])

-0.0016719117347637911


**bm_list_dicos_tfidf_sort** représente une double liste de couple où une liste interne correspond à un document et contient des couples où le premier élément est un mot et le deuxième élément est la valeur **tfidf** de ce mot.

Pour un document donné, la liste interne correspondante a les couples ordonnés de manière décroissante selon les valeurs tfidf.

In [59]:
bm_list_dicos_tfidf_sort = [sorted(dic.items(), key=operator.itemgetter(1), reverse=True) for dic in bm_list_dicos_tfidf]
bm_list_dicos_tfidf_sort[0][0:5]

[('trains', 0.0040942072869618994),
 ('fortuit', 0.0040942072869618994),
 ('héliportée', 0.0027294715246412664),
 ('roulante', 0.0027294715246412664),
 ('mordu', 0.0027294715246412664)]

On affiche l'état actuel.

In [60]:
cats = list(dico_flat.keys())
#for i,dic in enumerate(bm_list_dicos_tfidf_sort):
for i,dic in enumerate(bm_list_dicos_f):    
    print('Le nombre de mots distincts pour la catégorie ', cats[i], ' :', len(dic))

Le nombre de mots distincts pour la catégorie  ACCIDENT  : 471
Le nombre de mots distincts pour la catégorie  ACCIDENT - DE TRAVAIL  : 953
Le nombre de mots distincts pour la catégorie  ACCIDENT - DE CIRCULATION - AVEC ANIMAL  : 1718
Le nombre de mots distincts pour la catégorie  ACCIDENT - DE CIRCULATION - AVEC BLESSE  : 2036
Le nombre de mots distincts pour la catégorie  INONDATION  : 996


Finalement, on garde comme définition pour chaque catégorie une liste avec au maximum 500 mots les plus fréquents (dans le sens de la valeur tfidf) qui sont apparus dans les exposés des faits associés à la catégorie.

**dico_def_categories_sans_doublon** est un dictionnaire où les clés sont les noms des catégories et les valeurs sont des listes avec les 100 mots - uniques sans doublon - les plus fréquents correspondant à la "définition" des catégories.

In [61]:
cats = list(dico_flat.keys())
dico_def_categories_sans_doublon = {}
for i,li in enumerate(bm_list_dicos_tfidf_sort):
    lis = []
    for j,elem in enumerate(li):
        if j < 500:
            lis.append(elem[0])
    dico_def_categories_sans_doublon[cats[i]]=lis
print(list(dico_def_categories_sans_doublon.values())[0][0:15])

['trains', 'fortuit', 'héliportée', 'roulante', 'mordu', 'trouvant', 'échelle', 'envolée', 'acheminent', 'trolley-bus', 'lima', 'ambulatoire', 'domestique', 'graves', 'lidle']


**dico_def_categories** est un dictionnaire où les clés sont les noms des catégories et les valeurs sont des listes avec les 100 mots les plus fréquents correspondant à la "définition" des catégories mais apparaissant autant de fois qu'au début.

Donc, **dico_def_categories** correspond au **dico_flat** où dans chaque liste correspondant à une définition on a gardé que les mots les plus fréquents.

**corpus_def** est une liste de listes dans laquelle chaque liste interne contient les mots les plus fréquents de la définition d'une catégorie.

In [62]:
dico_def_categories = {}
corpus_def = []
for key,value in dico_flat.items():
    new_value = [elem for elem in value if elem in dico_def_categories_sans_doublon[key]]
    dico_def_categories[key] = new_value
    corpus_def.append(new_value)
print(list(dico_def_categories.values())[0][0:15])
print(corpus_def[0][0:15])

['fille', 'tombé', 'sol', 'saigne', 'bouche', 'précisions', 'femme', 'blessée', "s'est", 'envolée', 'ambulance', 'place', 'acheminent', 'blessée', "l'hôpital"]
['fille', 'tombé', 'sol', 'saigne', 'bouche', 'précisions', 'femme', 'blessée', "s'est", 'envolée', 'ambulance', 'place', 'acheminent', 'blessée', "l'hôpital"]


## Prédiction de la "bonne" catégorie

Au début, on procède comme dans le chapitre d'avant sauf qu'on utilise à la place de la variable **corpus**, la variable **corpus_def** (qui est la liste des listes de mots des définitions des catégories).

In [63]:
bm_def = bm25.BM25(corpus_def)

In [64]:
print(bm_def.corpus_size)

5


In [65]:
print(bm_def.avgdl)

2206.8


In [66]:
bm_def_list_dicos_f = bm_def.f
bm_def_list_dicos_f[0]['trains']

3

In [67]:
bm_def_dico_df = bm_def.df
bm_def_dico_df['trains']

1

In [68]:
bm_def_dico_idf = bm_def.idf
bm_def_dico_idf['trains']

1.0986122886681098

In [69]:
bm_def_average_idf = sum(bm_def_dico_idf.values())/len(bm_def_dico_idf.values())
print(bm_def_average_idf)

1.0704677966076261


On définit la fonction ad-hoc **clean** qui reçoit comme argument un exposé des faits (donc une string formée de plusieurs mots) et qui retourne la liste des mots de l'exposé des faits mais sans les "mots sans intérêt".

In [70]:
def nettoyer(expo):
    spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS
    blob = tb(expo.lower())
    tokens = list(blob.words)
    tokens = [elem for elem in tokens if elem not in spacy_stop_words]
    tokens = [elem for elem in tokens if not any(c.isdigit() for c in elem)]
    return tokens

**expo_new_text** est un nouvel exposé (donné normalement comme un texte avec des mots) "à classer", c'est-à-dire pour lequel il faut prédire la bonne définition.

In [71]:
#expo_new_text = dico['ACCIDENT'][3]
#expo_new_text = dico['ACCIDENT - DE TRAVAIL'][3]
expo_new_text = dico['ACCIDENT - DE CIRCULATION - AVEC ANIMAL'][9]
#expo_new_text = dico['ACCIDENT - DE CIRCULATION - AVEC BLESSE'][3]
#expo_new_text = dico['INONDATION'][3]
expo_new_text

'A heurté plusieurs sangliers. 1 animal DCD et beaucoup de débris sur la chaussée. '

A partir de l'exposé **expo_new_text**, on obtient **expo_new** qui est la liste de ces mots pertinents (sans les "mots sans intérêt")

In [72]:
expo_new = nettoyer(expo_new_text)
expo_new

['heurté', 'sangliers', 'animal', 'dcd', 'débris', 'chaussée']

**bm_def.get_score** donne le score d'un certain document $D$ fourni comme premier argument par rapport au document $Q$ qui a dans le corpus l'indice précisé comme deuxième argument et pour une moyenne IDF du corpus indiqué comme troisième argument

$$score(D,Q) = \sum_{i=1}^{n} {IDF(mot_i)}\cdot{\frac{f(mot_i,D)\cdot(k_1+1)}{f(mot_i,D)+k_1\cdot(1-b+b\cdot\frac{len(D)}{avgdl})}}$$

où :
- le document $Q$ a $n$ mots et il est le document par rapport auquel on calcule le score du document $D$
- $mot_i$ est un mot courant du document $Q$ par rapport auquel on calcule le score du document $D$
- $k_1\in [1.2,2.0]$
- $b=0.75$

Concrètement, on obtient ci-dessous le score (de similitude) du "expo_new" par rapport au document (en fait la définition de la catégorie) d'indice 0 du corpus

In [73]:
print(bm_def.get_score(expo_new, 1, bm_def_average_idf))

0


**bm_def.get_scores** retourne la liste des scores d'un certain document fourni comme premier argument par rapport à **tous** les documents du corpus, pour une moyenne IDF du corpus indiqué comme deuxième argument

In [74]:
scores = bm_def.get_scores(expo_new, bm_def_average_idf)
scores

[3.7150344628285166, 0, 7.4757335298424845, 0.5604346941973094, 0]

**bm25.get_bm25_weights(corpus_def)** retourne les scores (les poids) des documents dans le corpus sous la forme d'une liste de N listes internes contenant chacune N scores

La liste interne i contient les N scores du document i par rapport aux N documents du corpus (y compris par rapport à lui-même).

On constate que le score de chaque définition par rapport à elle-même est très élevé tandis que les scores d'une définition par rapport aux autres définitions sont assez réduites ce qui montre que les définitions sont "perpendiculaires" entre elles (ont peu de mots communs entre elles).

In [75]:
poids = bm25.get_bm25_weights(corpus_def)
print("Les scores\n", poids)

Les scores
 [[1541.0843138916189, 7.7198633166389925, 2.7860309415594737, 7.00441217219824, 5.978077461764558], [29.335283890967634, 1497.528286837334, 2.7346088384234113, 110.96980441842611, 4.862331043949091], [29.68241975004199, 6.754802454556558, 7346.208335293282, 1052.8119085766361, 7.574168770127952], [87.7394947803794, 296.7533389843918, 521.9579668198437, 4903.671816404552, 6.460568301829697], [35.20686357039076, 16.578710527924144, 6.208170749246817, 8.166765352642342, 3353.1235295792058]]


Le score maximal du nouvel exposé (qui peut être obtenu par plusieurs documents, i.e. par plusieurs définitions de catégories).

In [76]:
score_max = max(scores)
score_max

7.4757335298424845

On vérifie s'il y a plusieurs documents (définitions de catégorie) qui obtiennent le score maximal

In [77]:
[print(str(score)) for score in scores if score==score_max]

7.4757335298424845


[None]

La liste avec les indices des documents (définitions de catégories) qui ont obtenu le score maximal

In [78]:
indexes_max = np.argwhere(np.asarray(scores) == score_max).flatten().tolist()
indexes_max

[2]